# 透视表和交叉表

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

## df.pivot_table - 透视表

其实就是，行转列操作（注意这里还可以根据参数index来确定分组）

pivot_table的参数：

- `values`：待聚合的列的名称，默认聚合所有数值列。
- `index`：用于分组的列名或其他分组键，出现在结果透视表的行。
- `columns`：用于分组的列名或其他分组键，出现在结果透视表的列。
- `aggfunc`：聚合函数或函数列表，默认为mean。可以是任何对groupby有效的函数。
- `margins`：添加行/列小计和总结，默认为False。
- `margins_name`：需要添加总结的行/列名称，默认为All（如果margins为True）。

作用：把某些列作为维度之后，重新调整表的构造。

而，交叉表，是在透视表的基础上面，又对某列里的数据进行展开，重新调整表的构造。

In [2]:
tips = pd.read_csv('data/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [3]:
tips.pivot_table(index=['time', 'smoker']) # 使用time和smoker作为index创建透视表。
# cc：默认为mean，详情见上文参数描述；

size       tip   tip_pct  total_bill
time   smoker                                          
Dinner No      2.735849  3.126887  0.158653   20.095660
       Yes     2.471429  3.066000  0.160828   21.859429
Lunch  No      2.511111  2.673778  0.160920   17.050889
       Yes     2.217391  2.834348  0.170404   17.399130

In [4]:
# cc：只要再reset_index()，就成一个“友好”表格了。
tips.pivot_table(index=['time', 'smoker']).reset_index()


,time,smoker,size,tip,tip_pct,total_bill
0,Dinner,No,2.735849,3.126887,0.158653,20.095660
1,Dinner,Yes,2.471429,3.066000,0.160828,21.859429
2,Lunch,No,2.511111,2.673778,0.160920,17.050889
3,Lunch,Yes,2.217391,2.834348,0.170404,17.399130


In [5]:
# values代表要保留的列
# index代表把哪些列变成行索引
# columns表示针对哪些列进行汇总统计
# margins=True表示做分项小计，这个是主要针对 No,Yes 这两列做小计，且聚合函数默认为 np.mean
# 另外还有个变量margins_name,默认为All。现在指定Smoker列。
tips.pivot_table(values=['tip_pct', 'size'],
                 index=['time', 'day'],
                 columns=['smoker'],
                 margins=True,
                 margins_name='Smoker')

size                       tip_pct                    
smoker             No       Yes    Smoker        No       Yes    Smoker
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
Smoker       2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [5]:
# 小计 和 透视列 的聚合方式是一样的。
tips.pivot_table(values=['tip_pct', 'size'],
                 index=['time', 'day'],
                 columns=['smoker'],
                 aggfunc='sum',
                 margins=True,
                 margins_name='Smoker')
# 问题：有没有 透视列聚合方式为 np.mean，小计列聚合方式为 np.sum ？

size                  tip_pct                      
smoker          No    Yes Smoker         No        Yes     Smoker
time   day                                                       
Dinner Fri     6.0   20.0     26   0.418867   1.488126   1.906993
       Sat   115.0  104.0    219   7.112145   6.212055  13.324199
       Sun   167.0   49.0    216   9.126438   3.557756  12.684194
       Thur    2.0    NaN      2   0.159744        NaN   0.159744
Lunch  Fri     3.0   11.0     14   0.187735   1.133620   1.321354
       Thur  110.0   40.0    150   7.053669   2.785676   9.839345
Smoker       403.0  224.0    627  24.058598  15.177232  39.235830

In [6]:
tips.pivot_table(values='tip_pct', # 被统计对象
                 index=['time', 'smoker'],
                 columns='day',
                 aggfunc=np.mean,
                 margins=True) 

day                 Fri       Sat       Sun      Thur       All
time   smoker                                                  
Dinner No      0.139622  0.158048  0.160113  0.159744  0.158653
       Yes     0.165347  0.147906  0.187250       NaN  0.160828
Lunch  No      0.187735       NaN       NaN  0.160311  0.160920
       Yes     0.188937       NaN       NaN  0.163863  0.170404
All            0.169913  0.153152  0.166897  0.161276  0.160803

In [6]:
tips.pivot_table(values = 'size',
                 index=['time', 'smoker'],
                 columns='day',
                 aggfunc='sum',
                 fill_value=0) # 缺失值用0填充

day            Fri  Sat  Sun  Thur
time   smoker                     
Dinner No        6  115  167     2
       Yes      20  104   49     0
Lunch  No        3    0    0   110
       Yes      11    0    0    40

## pd.crosstab - 交叉表

在x ，y轴都有展开

参数：
- index,
- columns,
- values=None,
- rownames=None,
- colnames=None,
- aggfunc=None,
- margins=False,
- margins_name: str = 'All',
- dropna: bool = True,
- normalize=False,

In [7]:
sample = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
gender = ['F', 'M', 'F', 'M', 'M', 'M', 'F', 'F', 'M', 'F']
handedness = ['R', 'L', 'R', 'R', 'L', 'R', 'R', 'L', 'R', 'R']
data = pd.DataFrame({'Sample': sample, 'Gender': gender, 'Handedness': handedness},
                    columns=['Sample', 'Gender', 'Handedness'])
data

,Sample,Gender,Handedness
0,1,F,R
1,2,M,L
2,3,F,R
3,4,M,R
4,5,M,L
5,6,M,R
6,7,F,R
7,8,F,L
8,9,M,R
9,10,F,R


In [8]:
pd.crosstab(data.Gender, data.Handedness , margins=True) # 根据性别和手向生成交叉表，分别汇总统计数量。
# 看来聚合方式的默认参数为'count'

# cc：发现在这里只有 Gender 和 Handedness 参与构建，sample字段直接没有。

Handedness,L,R,All
Gender,,,
F,1,4,5
M,2,3,5
All,3,7,10


In [14]:
# cc：如果用透视表结果怎样？

data.pivot_table(
    index = ['Gender'],
    columns = ['Handedness'],
    aggfunc = 'count',
    margins = True
                )

# cc：但是在这里输出看到，sample字段还是在的。变成了3级索引，这么来看还是crosstab好用一点；

Sample       
Handedness      L  R All
Gender                  
F               1  4   5
M               2  3   5
All             3  7  10

In [21]:
pd.crosstab(
    index,
    columns,
    values=None,
    rownames=None,
    colnames=None,
    aggfunc=None,
    margins=False,
    margins_name: str = 'All',
    dropna: bool = True,
    normalize=False,
) -> 'DataFrame'

In [23]:
# 行索引是time/day
# 针对smoke做交叉表，列名是smoker，因为margins=True，所以统计all。
ct = pd.crosstab(index=[tips.time, tips.day], 
                 columns = tips.smoker, 
                 values= tips.size,
                 aggfunc = 'count',
                 margins=True)
ct
# 注意，crosstab的参数传入的都会Series 类型，也就是要把df.column 传进去；
# 而 pivot_table 只要传字段名即可；
# 因为 df.pivot_table, pd.crosstab，如果用df.crosstab 会报错：'DataFrame' object has no attribute 'crosstab'

smoker          No   Yes  All
time   day                   
Dinner Fri     3.0   9.0   12
       Sat    45.0  42.0   87
       Sun    57.0  19.0   76
       Thur    1.0   NaN    1
Lunch  Fri     1.0   6.0    7
       Thur   44.0  17.0   61
All          151.0  93.0  244

In [34]:
# cc：使用透视表：
ct2 = tips.pivot_table(
    values = ['size'],
    index = ['time', 'day'],
    columns = ['smoker'],
    aggfunc = 'count',
    margins = True
)

ct2

# cc：使用透视表虽然可以实现类似交叉表的结构，但是在维度上可以看到，透视表还多了一个维度。（如下size）
# 而交叉表不存在这个问题。

size           
smoker          No   Yes  All
time   day                   
Dinner Fri     3.0   9.0   12
       Sat    45.0  42.0   87
       Sun    57.0  19.0   76
       Thur    1.0   NaN    1
Lunch  Fri     1.0   6.0    7
       Thur   44.0  17.0   61
All          151.0  93.0  244